# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle
import numpy as np

# TODO: Fill this in based on where you saved the training and testing data

training_file = './traffic-signs-data/train.p'
validation_file='./traffic-signs-data/valid.p'
testing_file = './traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [2]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: Number of testing examples.
n_valid = X_valid.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:3]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 34799
Number of testing examples = 12630
Image data shape = (32, 32)
Number of classes = 43


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

cols = 6
rows = int(np.floor(n_classes/cols)) + 1

label_list = sorted(list(set(y_train)))
label_count = [np.sum((y_train == label)) for label in label_list]

f, axis = plt.subplots(nrows=rows, ncols=cols,figsize=image_shape)
axis = axis.reshape(-1)
for idx in range(len(label_list)):
    label = label_list[idx]
    img = X_train[y_train == label][10]
    axis[idx].imshow(img.astype(np.uint8),cmap='gray')
    axis[idx].set_title('Class {}, count {}'.format(label,label_count[idx]),size=18)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [3]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.
import tensorflow as tf
from sklearn.utils import shuffle
import cv2,os, shutil

In [4]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
def localContrastNormalizationYchannel(img):
    img = cv2.cvtColor(img,cv2.COLOR_RGB2YUV)
    img[...,0] = clahe.apply(img[...,0])
    return cv2.cvtColor(img,cv2.COLOR_YUV2RGB)

for idx in range(n_train):
    X_train[idx] = localContrastNormalizationYchannel( X_train[idx])
for idx in range(n_valid):
    X_valid[idx] = localContrastNormalizationYchannel( X_valid[idx])
for idx in range(n_test):
    X_test[idx] = localContrastNormalizationYchannel( X_test[idx])

In [5]:
def randomWarpAffineWithColorPerm(img):
    shift = 16 + np.random.randint(low=-5, high=5,size=[2])
    rotatingAngle = np.random.randint(low=-16, high=16)
    scale = 0.8 + np.random.random()*0.4
    affine_mat = cv2.getRotationMatrix2D((shift[0],shift[1]),rotatingAngle,scale)
    out = cv2.warpAffine(img,affine_mat,(32,32)) + np.random.randint(low=-10, high=10,size=[32,32,3])
    return out.astype(np.uint8)
aumentedX = np.zeros(shape=[0,32,32,3])
aumentedY = np.zeros(shape=[0])
for idx in range(7):
    newXTrain = np.array([randomWarpAffineWithColorPerm(X_train[idx]) for idx in range(n_train)])
    newYTrain = y_train.copy()
    aumentedX = np.vstack((aumentedX,newXTrain))
    aumentedY = np.hstack((aumentedY,newYTrain))

In [6]:
X_train = np.vstack((X_train,aumentedX))
y_train = np.hstack((y_train,aumentedY))

### Model Architecture

In [7]:
def ConvNet(x, num_of_classes,keepProb, name = 'ConvNet'):
    with tf.name_scope(name):
        # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
        mu = 0
        sigma = 0.1

        # TODO: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x12.
        with tf.name_scope('conv1'):
            w1 = tf.Variable(tf.truncated_normal([5,5,3,12], mean=mu, stddev=sigma),name='w')
            b1 = tf.Variable(tf.zeros([12]),name='b')
            logits = tf.nn.conv2d(x, w1, [1,1,1,1], padding='VALID', name='conv')
            logits = tf.nn.bias_add(logits,b1,name='addBias')

            # TODO: Activation.
            logits = tf.nn.relu(logits,name='relu')

            # TODO: Pooling. Input = 28x28x12. Output = 14x14x12.
            logits = tf.nn.max_pool(logits, [1,2,2,1], [1,2,2,1], padding='VALID', name='pool')
            
        # TODO: Layer 2: Convolutional. Output = 10x10x24.
        with tf.name_scope('conv2'):
            w2 = tf.Variable(tf.truncated_normal([5,5,12,24], mean=mu, stddev=sigma),name='w')
            b2 = tf.Variable(tf.zeros([24]),name='b')
            logits = tf.nn.conv2d(logits, w2, [1,1,1,1], padding='VALID', name='conv')
            logits = tf.nn.bias_add(logits,b2,name='addBias')

            # TODO: Activation.
            logits = tf.nn.relu(logits,name='relu')

            # TODO: Pooling. Input = 10x10x24. Output = 5x5x24.
            logits = tf.nn.max_pool(logits, [1,2,2,1], [1,2,2,1], padding='VALID', name='pool')
        
        # TODO: Flatten. Input = 5x5x24. Output = 600.
        with tf.name_scope('Flatten'):
            branch1 = tf.contrib.layers.flatten(logits)
            
        # TODO: Layer 2: Convolutional. Output = 3x3x48.
        with tf.name_scope('conv3'):
            w6 = tf.Variable(tf.truncated_normal([3,3,24,48], mean=mu, stddev=sigma),name='w')
            b6 = tf.Variable(tf.zeros([48]),name='b')
            logits = tf.nn.conv2d(logits, w6, [1,1,1,1], padding='VALID', name='conv')
            logits = tf.nn.bias_add(logits,b6,name='addBias')

            # TODO: Activation.
            logits = tf.nn.relu(logits,name='relu')

            # TODO: Pooling. Input = 4x4x48. Output = 2x2x48.
            #logits = tf.nn.max_pool(logits, [1,2,2,1], [1,2,2,1], padding='VALID', name='pool')

        # TODO: Flatten. Input =3x3x48. Output = 432.
        with tf.name_scope('Flatten'):
            logits = tf.contrib.layers.flatten(logits)
        
        # TODO: Output = 1056.
        logits = tf.concat([branch1,logits],1,name='concat')

        # TODO: Layer 3: Fully Connected. Input = 1032. Output = 512.
        with tf.name_scope('fc1'):
            w3 = tf.Variable(tf.truncated_normal([1032,512], mean=mu, stddev=sigma),name='w')
            b3 = tf.Variable(tf.zeros([512]),name='b')
            logits = tf.add(tf.matmul(logits,w3,name='mul'),b3,name='addBias')

            # TODO: Activation.
            logits = tf.nn.relu(logits,name='relu')
        
        #logits = tf.nn.dropout(logits, keep_prob=keepProb,name='dropout')

        # TODO: Layer 4: Fully Connected. Input = 512. Output = 256.
        with tf.name_scope('fc2'):
            w4 = tf.Variable(tf.truncated_normal([512,256], mean=mu, stddev=sigma),name='w')
            b4 = tf.Variable(tf.zeros([256]),name='b')
            logits = tf.add(tf.matmul(logits,w4,name='mul'),b4,name='addBias')

            # TODO: Activation.
            logits = tf.nn.relu(logits,name='relu')
            

        # TODO: Layer 5: Fully Connected. Input = 256. Output = num_of_classes.
        with tf.name_scope('fc3'):
            w5 = tf.Variable(tf.truncated_normal([256,num_of_classes], mean=mu, stddev=sigma),name='w')
            b5 = tf.Variable(tf.zeros([num_of_classes]),name='b')
            logits = tf.add(tf.matmul(logits,w5,name='mul'),b5,name='addBias')

        return logits

## Input Data: x, y

In [8]:
with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32,[None,32,32,3],name='InputData')
    y = tf.placeholder(tf.int32,[None],name='InputLabel')
    dropoutKeepProb = tf.placeholder(tf.float32,name='dropoutKeepProb')
    y_one_hot = tf.one_hot(y,n_classes)

## Training pipeline

In [9]:
logits = ConvNet(x,n_classes,dropoutKeepProb)

with tf.name_scope('CrossEntropyLoss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot,name='cross_entropy')
    cross_entropy_loss = tf.reduce_mean(cross_entropy,name='cross_entropy_loss')
    #reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #reg_constant = 0.001  # Choose an appropriate one.
    #loss = cross_entropy_loss #+ reg_constant * tf.reduce_sum(reg_losses)
loss_on_trainset_summary = tf.summary.scalar('CrossEntropyLoss', cross_entropy_loss)

with tf.name_scope('Accuracy'):
    predicted_y_one_hot = tf.nn.softmax(logits,name='predicted_y_one_hot')
    correct_prediction = tf.equal(tf.argmax(predicted_y_one_hot, 1), tf.argmax(y_one_hot, 1))
    accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    

## Evaluation

In [10]:
def evaluate(X_data, y_data, sess, BATCH_SIZE):
    num_examples = len(X_data)
    total_accuracy = 0.
    total_loss = 0.
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, dropoutKeepProb:1.0})
        total_accuracy += (accuracy * len(batch_x))
    total_accuracy /= num_examples
    sess.run(acc_on_valset.assign(total_accuracy))
    return total_accuracy

#merged_summary = tf.summary.merge_all()

with tf.name_scope('EvaluationOnValSet'):
    acc_on_valset = tf.Variable(0,dtype=tf.float32, trainable=False)
acc_on_valset_summary = tf.summary.scalar('Acc', acc_on_valset)

## Optimizer

In [11]:
with tf.name_scope('Trainer'):
    learning_rate = 0.00005
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [12]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

BATCH_SIZE = 32
EPOCHS = 1000
saver = tf.train.Saver()
lr = 0.001

if os.path.exists('ConvNetOnTrafficSignGraph'):
    shutil.rmtree('ConvNetOnTrafficSignGraph')
    
if os.path.exists('ConvNetOnTrafficSign'):
    shutil.rmtree('ConvNetOnTrafficSign')
os.mkdir('ConvNetOnTrafficSign')
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    train_writer = tf.summary.FileWriter('./ConvNetOnTrafficSignGraph/train', sess.graph)
    test_writer = tf.summary.FileWriter('./ConvNetOnTrafficSignGraph/test')
    print("Training...")
    print()
    count = 0
    max_validation_accuracy = 0.0
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
#             sess.run(optimizer, feed_dict={ x: batch_x, 
#                                             y: batch_y, 
#                                             dropoutKeepProb:0.5}
#                     )
            _, summary = sess.run([optimizer,loss_on_trainset_summary], feed_dict={ x: batch_x, 
                                                                          y: batch_y, 
                                                                          dropoutKeepProb:0.5})
            train_writer.add_summary(summary,global_step=count)
            count += 1
        train_writer.flush() 
        
        validation_accuracy = evaluate(X_valid, y_valid, sess, BATCH_SIZE)
        test_writer.add_summary(sess.run(acc_on_valset_summary),global_step=i)
        test_writer.flush()
        
        if (i%10 == 0):
            lr *= 0.1
        
        if (validation_accuracy > max_validation_accuracy):
            saver.save(sess, save_path='./ConvNetOnTrafficSign/Model',global_step=i)
            max_validation_accuracy = validation_accuracy
        
        print("EPOCH {} ...Validation Accuracy = {:.3f}...MaxAcc ={:.3f}".format(i+1, validation_accuracy, max_validation_accuracy))
    
    coord.request_stop()
    coord.join(threads)
    sess.close()


Training...

EPOCH 1 ...Validation Accuracy = 0.667...MaxAcc =0.667
EPOCH 2 ...Validation Accuracy = 0.815...MaxAcc =0.815
EPOCH 3 ...Validation Accuracy = 0.878...MaxAcc =0.878
EPOCH 4 ...Validation Accuracy = 0.909...MaxAcc =0.909
EPOCH 5 ...Validation Accuracy = 0.923...MaxAcc =0.923
EPOCH 6 ...Validation Accuracy = 0.934...MaxAcc =0.934
EPOCH 7 ...Validation Accuracy = 0.946...MaxAcc =0.946
EPOCH 8 ...Validation Accuracy = 0.940...MaxAcc =0.946
EPOCH 9 ...Validation Accuracy = 0.947...MaxAcc =0.947
EPOCH 10 ...Validation Accuracy = 0.950...MaxAcc =0.950
EPOCH 11 ...Validation Accuracy = 0.946...MaxAcc =0.950
EPOCH 12 ...Validation Accuracy = 0.953...MaxAcc =0.953
EPOCH 13 ...Validation Accuracy = 0.950...MaxAcc =0.953
EPOCH 14 ...Validation Accuracy = 0.951...MaxAcc =0.953
EPOCH 15 ...Validation Accuracy = 0.953...MaxAcc =0.953
EPOCH 16 ...Validation Accuracy = 0.959...MaxAcc =0.959
EPOCH 17 ...Validation Accuracy = 0.953...MaxAcc =0.959
EPOCH 18 ...Validation Accuracy = 0.960...Ma

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [1]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.
import tensorflow as tf

In [ ]:
tf.layers.

### Analyze Performance

In [4]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [6]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 